In [2]:
import pandas as pd

In [7]:

df = pd.read_csv("./01_raw_data/기온, 강수량.csv",encoding='cp949')

station_to_province = {
    '속초': '강원', '북춘천': '강원', '철원': '강원', '동두천': '경기', '파주': '경기',
    '대관령': '강원', '춘천': '강원', '백령도': '인천', '북강릉': '강원', '강릉': '강원',
    '동해': '강원', '서울': '서울', '인천': '인천', '원주': '강원', '울릉도': '경북',
    '수원': '경기', '영월': '강원', '충주': '충북', '서산': '충남', '울진': '경북',
    '청주': '충북', '대전': '대전', '추풍령': '충북', '안동': '경북', '상주': '경북',
    '포항': '경북', '군산': '전북', '대구': '대구', '전주': '전북', '울산': '울산',
    '창원': '경남', '광주': '광주', '부산': '부산', '통영': '경남', '목포': '전남',
    '여수': '전남', '흑산도': '전남', '완도': '전남', '고창': '전북', '순천': '전남',
    '홍성': '충남', '제주': '제주', '고산': '제주', '성산': '제주', '서귀포': '제주',
    '진주': '경남', '강화': '인천', '양평': '경기', '이천': '경기', '인제': '강원',
    '홍천': '강원', '태백': '강원', '정선군': '강원', '제천': '충북', '보은': '충북',
    '천안': '충남', '보령': '충남', '부여': '충남', '금산': '충남', '세종': '세종',
    '부안': '전북', '임실': '전북', '정읍': '전북', '남원': '전북', '장수': '전북',
    '고창군': '전북', '영광군': '전남', '김해시': '경남', '순창군': '전북', '북창원': '경남',
    '양산시': '경남', '보성군': '전남', '강진군': '전남', '장흥': '전남', '해남': '전남',
    '고흥': '전남', '의령군': '경남', '함양군': '경남', '광양시': '전남', '진도군': '전남',
    '봉화': '경북', '영주': '경북', '문경': '경북', '청송군': '경북', '영덕': '경북',
    '의성': '경북', '구미': '경북', '영천': '경북', '경주시': '경북', '거창': '경남',
    '합천': '경남', '밀양': '경남', '산청': '경남', '거제': '경남', '남해': '경남'
}

df['시도'] = df['지점명'].map(station_to_province)

cols_to_numeric = ['평균기온(°C)', '평균최고기온(°C)', '평균최저기온(°C)', '월합강수량(00~24h만)(mm)']
for col in cols_to_numeric:
    df[col] = pd.to_numeric(df[col], errors='coerce')

grouped = (
    df
    .dropna(subset=['시도'])
    .groupby(['일시', '시도'])[cols_to_numeric]
    .mean()
    .reset_index()
)

national = (
    grouped
    .groupby('일시')[cols_to_numeric]   
    .mean()                            
    .reset_index()
)
national['시도'] = '전국' 
national = national[['일시', '시도'] + cols_to_numeric]

grouped_all = pd.concat([grouped, national], ignore_index=True)
grouped_all[['year', 'month']] = grouped_all['일시'].str.split('-', expand=True)

grouped_all = grouped_all.drop(columns='일시')
grouped_all= grouped_all[['year',	'month','시도',	'평균기온(°C)',	'평균최고기온(°C)',	'평균최저기온(°C)',	'월합강수량(00~24h만)(mm)'	]]
grouped_all.to_csv("./02_interim_data/temperature_rain_data.csv", index=False, encoding='utf-8-sig')
grouped_all.head(50)


,year,month,시도,평균기온(°C),평균최고기온(°C),평균최저기온(°C),월합강수량(00~24h만)(mm)
0,2020,01,강원,0.728571,5.821429,-3.328571,85.335714
1,2020,01,경기,0.600000,6.060000,-3.860000,70.280000
2,2020,01,경남,3.978571,9.385714,-0.278571,106.492857
3,2020,01,경북,2.778571,7.642857,-1.214286,90.414286
4,2020,01,광주,4.600000,9.000000,1.200000,74.900000
5,2020,01,대구,3.800000,8.400000,0.100000,70.400000
6,2020,01,대전,2.700000,7.800000,-1.200000,78.500000
7,2020,01,부산,6.400000,10.900000,3.100000,127.700000
8,2020,01,서울,1.600000,5.900000,-1.700000,60.500000
9,2020,01,세종,2.100000,7.100000,-1.700000,81.300000
